<h1 align="center">Практичиское задание 2: Sparse Variational Dropout</h1>

В этом задании вам предстоит реализовать Sparse VD -- метод для разреживания нейронных сетей https://arxiv.org/abs/1701.05369  

In [7]:
import math
import torch
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

from logger import Logger
from torch.nn import Parameter
from torch.autograd import Variable
from torchvision import datasets, transforms

## Реализуем полносвязный Sparse VD слой

In [8]:
class LinearSVDO(nn.Module):
    def __init__(self, in_features, out_features, threshold, bias=True):
        super(LinearSVDO, self).__init__()
        # in_features int
        # out_features int 
        # threshold float
        self.in_features = in_features
        self.out_features = out_features
        self.threshold = threshold

        # =======================================
        # Создайте параметры модели -- объекты класса Parameter
        # W размера (out_features x in_features)
        # log_sigma размера (out_features x in_features)
        # bias размера (1, out_features)
        # =======================================
        self.W = Parameter(torch.rand((self.out_features, self.in_features)) - 0.5) * 2 / self.in_features ** 0.5
        self.log_sigma = Parameter(torch.zeros((self.out_features, self.in_features)) - 5)
        self.bias = Parameter(torch.zeros((1, self.out_features)))
        
        self.reset_parameters()

    def reset_parameters(self):
        # =======================================
        # Инициализируйте параметры модели
        # W -- нормальный случайный шум с центром в 0 и маленькой дисперсией
        # log_sigma -- маленьким значением ~ -5 
        # bias -- можно 0
        # =======================================   
        self.W = Parameter(torch.randn((self.out_features, self.in_features)))
        self.log_sigma = Parameter(torch.zeros((self.out_features, self.in_features)) - 5)
        self.bias = Parameter(torch.zeros((1, self.out_features)))
        
        stdv = 1. / math.sqrt(self.W.size(1))
        self.W.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
        self.log_sigma.data.uniform_(-5, -5)
    
    @property
    def log_alpha(self):
        return 2 * self.log_sigma - (self.W ** 2 + 1e-8).log()
        
    def forward(self, x):
        # =======================================
        # x: Variable containing: [torch.FloatTensor of size batch_size x in_features]
        # Return: type: Variable containing [torch.FloatTensor of size batch_size x out_features]
        # ----------------------------------------
        # Тут нужно написать forward шаг для Sparse VD слоя для минибатча объектов x
        # На этапе обучения: Вернуть семпл активаций с помощью Local Reparametrization Trick 
        # На этапе тестирования: Вернуть средние активации, посчитанные с обрезанными весами 
        # Правило обрезания весов: alpha_ij < self.threshold ====> w_ij = 0
        # ----------------------------------------
        # Клипинг alpha_ij, например torch.clamp(self.log_alpha, -10, 10) может улучшить стабильность 
        # Чтобы не встретить nan-ы используйте трюк log(a) = log(a + 1e-8)
        # ======================================= 
        
        n, _ = x.shape
        if self.training: # training
            eps = Variable(torch.randn((n, self.out_features))) 
            output = F.linear(x, self.W, self.bias) + \
            eps * F.linear(x ** 2, self.log_sigma.clamp(-10, 10).exp() ** 2) ** 0.5
        else:
            w = self.W * (self.log_alpha.exp() < (self.threshold + 1e-8)).float()
            output = F.linear(x, w, self.bias)
        return output
        
    def kl_reg(self):
        # =======================================
        # Вернуть суммарную KL дивергенцию для всех параметров слоя 
        # Return: Variable containing: [torch.FloatTensor of size 1]
        # =======================================
        k1 = 0.63576
        k2 = 1.87320
        k3 = 1.48695
        C = -k1
        kl = -(k1 * torch.sigmoid(k2 + k3 * self.log_alpha) - 0.5 * (1 + (-self.log_alpha).exp()).log() + C)
        return kl.sum(dim=0).sum(dim=0)

## Создадим простую архитектуру LeNet-300-100

In [9]:
class Net(nn.Module):
    def __init__(self, threshold):
        super(Net, self).__init__()
        self.fc1 = LinearSVDO(28*28, 300, threshold)
        self.fc2 = LinearSVDO(300,  100, threshold)
        self.fc3 = LinearSVDO(100,  10, threshold)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

## Загрузим MNIST

In [10]:
def get_mnist(batch_size):
    trsnform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)

    return train_loader, test_loader

## Определим новую функцию потерь SGVLB

In [11]:
class SGVLB(nn.Module):
    def __init__(self, net, train_size):
        super(SGVLB, self).__init__()
        self.train_size = train_size
        self.net = net

    def forward(self, output, target, kl_weight=1.0):
        # =======================================
        # output -- ответы модели для минибатча [torch.FloatTensor of size batch_size x 10]
        # target -- настоящие ответы  [torch.LongTensor of size batch_size]
        # kl_weight -- коэффициент на который нужно умножить kl дивергенцию, нужен для отжига (читай ниже)
        # Вернуть Variable с посчитанной SGVLB функцией потерь 
        # Используйте self.net.children() для обхода всех слоев модели
        # !!! Проверьте что множитель перед data term правильный !!!
        # Return: Variable containing: [torch.FloatTensor of size 1]
        # =======================================
        loss = Variable(torch.FloatTensor([0]))
        for layer in self.net.children():
            loss += kl_weight * layer.kl_reg()
        loss += torch.nn.CrossEntropyLoss(size_average=False)(output, target) * self.train_size / target.shape[0]
        return loss

In [12]:
epochs, batch_size, threshold = 100, 100, 3
model = Net(threshold)
optimizer = torch.optim.Adam(model.parameters())# Тут ваш любимый оптимизатор, адам -- хороший выбор
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.9) # Тут расписание шага обучения torch.optim.lr_scheduler
fmt = {'tr_los': '3.1e', 'te_loss': '3.1e', 'sp_0': '.3f', 'sp_1': '.3f', 'lr': '3.1e', 'kl': '.2f'}
logger = Logger('sparse_vd', fmt=fmt)

In [13]:
from time import time
# Для ускорения обучения используйте отжиг kl 
# Отжиг кл помогает избежать "плохих локальных оптимумов" и устроен так 
# kl_weight стартует с маленького значения и увеличивается до 1 с каким-то шагом
# а потом остается 1 до конца обучения 

train_loader, test_loader = get_mnist(batch_size)
sgvlb = SGVLB(model, len(train_loader.dataset))
step = 0.02 # шаг для увеличения kl_weight

# ===============
kl_weight = step # Начальное значение веса KL
# ===============
start = time()
print(time() - start)
for epoch in range(1, epochs + 1):
    scheduler.step()
    model.train()
    train_loss, train_acc = 0, 0 
    kl_weight = min(kl_weight+step, 1) 
    logger.add_scalar(epoch, 'kl', kl_weight)
    logger.add_scalar(epoch, 'lr', scheduler.get_lr()[0])
    for batch_idx, (data, target) in (enumerate(train_loader)):
        data, target = Variable(data).view(-1, 28*28), Variable(target)
        optimizer.zero_grad()
        
        output = model(data)
        pred = output.data.max(1)[1] 
        loss = sgvlb(output, target, kl_weight)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data.numpy()
        train_acc += np.sum(pred.numpy() == target.data.numpy())

    logger.add_scalar(epoch, 'tr_los', train_loss / len(train_loader.dataset))
    logger.add_scalar(epoch, 'tr_acc', train_acc / len(train_loader.dataset) * 100)
    
    
    model.eval()
    test_loss, test_acc = 0, 0
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = Variable(data, volatile=True).view(-1, 28*28), Variable(target)
        output = model(data)
        test_loss += sgvlb(output, target, kl_weight).data[0]
        pred = output.data.max(1)[1] 
        test_acc += np.sum(pred.numpy() == target.data.numpy())
        
    logger.add_scalar(epoch, 'te_loss', test_loss / len(test_loader.dataset))
    logger.add_scalar(epoch, 'te_acc', test_acc / len(test_loader.dataset) * 100)
    
    for i, c in enumerate(model.children()):
        if hasattr(c, 'kl_reg'):
            logger.add_scalar(epoch, 'sp_%s' % i, (c.log_alpha.data.numpy() > threshold).mean())
    
    logger.iter_info()
    print(time() - start)

5.14984130859375e-05
  epoch    kl       lr    tr_los    tr_acc    te_loss    te_acc    sp_0    sp_1    sp_2
-------  ----  -------  --------  --------  ---------  --------  ------  ------  ------
      1  0.04  1.0e-03   2.5e+02      92.3    1.3e+02      96.4   0.352   0.131     0.0
31.576680183410645
      2  0.06  1.0e-03   1.4e+02      96.7    1.1e+02      97.4   0.464   0.209     0.0
63.64189958572388
      3  0.08  1.0e-03   1.2e+02      97.4    1.0e+02      97.7   0.555   0.299     0.1
93.79713273048401
      4  0.10  1.0e-03   1.1e+02      97.8    9.9e+01      97.6   0.625   0.347     0.1
123.9317524433136
      5  0.12  1.0e-03   9.9e+01      97.9    9.3e+01      98.0   0.677   0.414     0.1
155.2494010925293
      6  0.14  9.0e-04   8.9e+01      98.2    8.6e+01      98.2   0.753   0.474     0.1
187.20657110214233
      7  0.16  9.0e-04   8.7e+01      98.2    8.8e+01      98.1   0.779   0.515     0.1
217.05747842788696
      8  0.18  9.0e-04   8.5e+01      98.3    8.6e+01     

KeyboardInterrupt: 

In [28]:
# Посмотрим во сколько раз у нас уменьшилось количество весов в первом слое 
# Тут хотим получить в 30+ раз меньше весов без падения качества на тесте

all_w, kep_w = 0, 0

for c in model.children():
    kep_w += (c.log_alpha.data.numpy() < 1.5).sum()
    all_w += c.log_alpha.data.numpy().size

print('keept weight ratio =', all_w/kep_w)

keept weight ratio = 47.30762395592678


In [27]:
for c in model.children():
    c.threshold = 1.5
test_loss, test_acc = 0, 0
model.eval()
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = Variable(data, volatile=True).view(-1, 28*28), Variable(target)
    output = model(data)
    test_loss += sgvlb(output, target, kl_weight).data[0]
    pred = output.data.max(1)[1] 
    test_acc += np.sum(pred.numpy() == target.data.numpy())
print('loss:', test_loss, 'acc:', test_acc)

loss: 876349.1577148438 acc: 9809


In [29]:
# Посмотрим какая компрессия получилась на диске
import scipy
import numpy as np
from scipy.sparse import csc_matrix, csc_matrix, coo_matrix, dok_matrix

row, col, data = [], [], []
M = list(model.children())[0].W.data.numpy()
LA = list(model.children())[0].log_alpha.data.numpy()

for i in range(300):
    for j in range(28*28):
        if LA[i, j] < 1.5:
            row += [i]
            col += [j]
            data += [M[i, j]]

Mcsr = csc_matrix((data, (row, col)), shape=(300, 28*28))
Mcsc = csc_matrix((data, (row, col)), shape=(300, 28*28))
Mcoo = coo_matrix((data, (row, col)), shape=(300, 28*28))

In [30]:
np.savez_compressed('M_w', M)
scipy.sparse.save_npz('Mcsr_w', Mcsr)
scipy.sparse.save_npz('Mcsc_w', Mcsc)
scipy.sparse.save_npz('Mcoo_w', Mcoo)

In [31]:
ls -lah | grep _w

-rw-rw-r-- 1 dvschelchkov dvschelchkov  22K Mar 28 23:57 Mcoo_w.npz
-rw-rw-r-- 1 dvschelchkov dvschelchkov  21K Mar 28 23:57 Mcsc_w.npz
-rw-rw-r-- 1 dvschelchkov dvschelchkov  21K Mar 28 23:57 Mcsr_w.npz
-rw-rw-r-- 1 dvschelchkov dvschelchkov 840K Mar 28 23:57 M_w.npz


## Бонусная часть баллов 25%

- Сжать этим методом сверточную сеть LeNet-5-Caffe в 100 + раз http://caffe.berkeleyvision.org/gathered/examples/mnist.html
- Поэкспериментировать с разной битностью весов -- при какой битности качество начинает падать
- Помогают ли веса меньшей битности сэкономить место на диске?

In [22]:
class Conv2d(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        self.log_sigma = Parameter(torch.Tensor(
                out_channels, in_channels // groups, *kernel_size))
        super(Conv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            False, _pair(0), groups, bias)

    @property
    def log_alpha(self):
        return 2 * self.log_sigma - (self.W ** 2 + 1e-8).log()
    
    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
        self.log_sigma.data.uniform_(-5, -5)

    def forward(self, input):
        eps = Variable(torch.randn((n, self.out_features, ))) 
        result = F.conv2d(input, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
        eps = Variable(torch.randn(result.shape)) 
        result += F.conv2d(input ** 2, self.log_sigma.clamp(-10, 10).exp() ** 2, self.stride,
                        self.padding, self.dilation, self.groups) ** 0.5
        return result
        
    def kl_reg(self):
        # =======================================
        # Вернуть суммарную KL дивергенцию для всех параметров слоя 
        # Return: Variable containing: [torch.FloatTensor of size 1]
        # =======================================
        k1 = 0.63576
        k2 = 1.87320
        k3 = 1.48695
        C = -k1
        kl = -(k1 * torch.sigmoid(k2 + k3 * self.log_alpha) - 0.5 * (1 + (-self.log_alpha).exp()).log() + C)
        return kl.sum(dim=0).sum(dim=0)

In [ ]:
class Net(nn.Module):
    def __init__(self, threshold):
        super(Net, self).__init__()
        self.fc1 = LinearSVDO(28*28, 300, threshold)
        self.fc2 = LinearSVDO(300,  100, threshold)
        self.fc3 = LinearSVDO(100,  10, threshold)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

In [ ]:
epochs, batch_size, threshold = 100, 100, 3
model = Net(threshold)
optimizer = torch.optim.Adam(model.parameters())# Тут ваш любимый оптимизатор, адам -- хороший выбор
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1000000) # Тут расписание шага обучения torch.optim.lr_scheduler
fmt = {'tr_los': '3.1e', 'te_loss': '3.1e', 'sp_0': '.3f', 'sp_1': '.3f', 'lr': '3.1e', 'kl': '.2f'}
logger = Logger('sparse_vd', fmt=fmt)